In [2]:
import math
import os
import subprocess
from glob import glob
from itertools import combinations
from urllib.error import URLError

import earthpy as et
import earthpy.appeears as etapp
import geopandas as gpd
import holoviews as hv
import hvplot as hv
import hvplot.pandas
import hvplot.xarray
import laspy
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pdal
import pylas
import requests
import rasterio
import rioxarray as rxr
import rioxarray.merge as rxrmerge
import skfuzzy as fuzz
from skfuzzy import control as ctrl
from IPython.display import Image
import xarray as xr
import xrspatial
import warnings
import zipfile

from osgeo import gdal, gdal_array, osr
from rasterio.transform import from_origin
from scipy.interpolate import griddata

In [ ]:
### Pseudocode for process

import project area shapefile

import LIDAR index grid

intersect index grid and project areas shapefile to identify tiles to download

for each project area:
* download tiles
* process tiles with LASTools into canopy height dem
* clip to project area
* merge if necessary

Need to install PDAL, this requires installing visual studio build tools:
    
https://visualstudio.microsoft.com/visual-cpp-build-tools/

then run pip install pdal

may need to install cmake from here too:

https://cmake.org/download/

In [2]:
# Set up directory
data_dir = os.path.join(et.io.HOME, et.io.DATA_NAME)
project_dir = os.path.join(data_dir, "treebeard")
# Create the directory if it doesn't exist
os.makedirs(data_dir, exist_ok=True)

las_index_path = os.path.join(
    data_dir,
    'earthpy-downloads',
    'lidar_index_cspn_q2',
    'lidar_index_cspn_q2.shp'
)

# Download LIDAR index tiles
if not os.path.exists(las_index_path):
    las_index_url = ('https://gisdata.drcog.org:8443/geoserver/DRCOGPUB/'
             'ows?service=WFS&version=1.0.0&request=GetFeature&'
             'typeName=DRCOGPUB:lidar_index_cspn_q2&outputFormat=SHAPE-ZIP')

    las_index_shp = et.data.get_data(url=las_index_url)

las_index_gdf = (
    gpd.read_file(las_index_path).set_index('tile')
#    .loc[['N3W345']]
)

las_index_gdf = las_index_gdf.to_crs('EPSG:4269')

las_index_plot = las_index_gdf.hvplot(
    tiles = 'OSM',
    geo = True,
    line_color='black',
    line_width=2,
    fill_alpha=0
)
las_index_plot

:Overlay
   .WMTS.I     :WMTS   [Longitude,Latitude]
   .Polygons.I :Polygons   [Longitude,Latitude]

In [4]:
# Open project areas shapefile
proj_zip_path = 'assets/project_areas_merged.zip'

with zipfile.ZipFile(proj_zip_path, 'r') as zip_ref:
    temp_dir = '/tmp/extracted_shapefile'  # You can specify any temporary directory
    zip_ref.extractall(temp_dir)
    
extracted_shapefile_path = temp_dir + '/'

proj_area_gdf = gpd.read_file(extracted_shapefile_path)

proj_area_gdf = proj_area_gdf.to_crs("EPSG:4326")

proj_area_plot = proj_area_gdf.hvplot(
    x='x',
    y='y',
    aspect='equal',
    tiles='EsriImagery',
    geo=True,
    line_color='blue',
    line_width=2,
    fill_alpha=0
)

proj_area_plot


:Overlay
   .WMTS.I     :WMTS   [Longitude,Latitude]
   .Polygons.I :Polygons   [x,y]

In [5]:
# Identify the tiles that intersect each project area
select_tiles_gdf = gpd.sjoin(las_index_gdf, proj_area_gdf, how='inner', op='intersects')

select_tiles_gdf.reset_index(drop=False)
select_tiles_gdf.hvplot(
    x='x',
    y='y',
    aspect='equal',
    tiles='EsriImagery',
    geo=True,
    line_color='blue',
    line_width=2,
    fill_alpha=0
)

C:\Users\Pete\miniconda3\envs\earth-analytics-python\lib\site-packages\IPython\core\interactiveshell.py:3488: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):
C:\Users\Pete\AppData\Local\Temp\ipykernel_23232\1148491686.py:2: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: EPSG:4269
Right CRS: EPSG:4326

  select_tiles_gdf = gpd.sjoin(las_index_gdf, proj_area_gdf, how='inner', op='intersects')


:Overlay
   .WMTS.I     :WMTS   [Longitude,Latitude]
   .Polygons.I :Polygons   [x,y]

In [6]:
select_tiles_gdf = select_tiles_gdf.reset_index(drop=False)
select_tiles_gdf

,tile,gid,area,storage,geometry,index_right,Shape_Leng,Shape_Area,Acreage,Proj_ID
0,N4W264,191,CSPN_Q2,lidararchive,"POLYGON ((-105.27729 40.21980, -105.29620 40.2...",0,806.343609,32945.419705,0.0,Unnamed 1
1,N4W351,761,CSPN_Q2,lidararchive,"POLYGON ((-105.52309 40.23450, -105.54201 40.2...",1,0.017313,0.000015,0.0,Zumwinkel
2,N4W399,993,CSPN_Q2,lidararchive,"POLYGON ((-105.37191 40.17646, -105.39080 40.1...",3,0.000000,0.000000,0.0,Conifer Hill
3,N4W397,1090,CSPN_Q2,lidararchive,"POLYGON ((-105.40970 40.17649, -105.42860 40.1...",3,0.000000,0.000000,0.0,Conifer Hill
4,N4W389,1405,CSPN_Q2,lidararchive,"POLYGON ((-105.37188 40.19095, -105.39078 40.1...",3,0.000000,0.000000,0.0,Conifer Hill
5,N4W396,1593,CSPN_Q2,lidararchive,"POLYGON ((-105.42860 40.17651, -105.44749 40.1...",3,0.000000,0.000000,0.0,Conifer Hill
6,N4W388,1712,CSPN_Q2,lidararchive,"POLYGON ((-105.39078 40.19097, -105.40968 40.1...",3,0.000000,0.000000,0.0,Conifer Hill
7,N4W290,1787,CSPN_Q2,lidararchive,"POLYGON ((-105.35301 40.17643, -105.37191 40.1...",3,0.000000,0.000000,0.0,Conifer Hill
8,N4W398,1872,CSPN_Q2,lidararchive,"POLYGON ((-105.39080 40.17648, -105.40970 40.1...",3,0.000000,0.000000,0.0,Conifer Hill
9,N3W308,1978,CSPN_Q2,lidararchive,"POLYGON ((-105.39083 40.16198, -105.40972 40.1...",3,0.000000,0.000000,0.0,Conifer Hill


In [7]:
# Generate list of all tiles per project area
tiles_by_area = select_tiles_gdf.groupby('Proj_ID')['tile'].apply(list).reset_index()
tiles_by_area

,Proj_ID,tile
0,Conifer Hill,"[N4W399, N4W397, N4W389, N4W396, N4W388, N4W29..."
1,Unnamed 1,[N4W264]
2,Unnamed 2,"[N4W381, N4W391]"
3,Zumwinkel,[N4W351]


In [ ]:
# Process tiles for each project area

# Set this to your LASTools directory after installing
lastools_path = "C:\\Users\\Pete\\Desktop\\GIS\\LAStools\\bin\\las2dem.exe"

las_root_url = 'https://lidararchive.s3.amazonaws.com/2020_CSPN_Q2/'
proj_dict = {}
for index, row in tiles_by_area.iterrows():
    tiles = row['tile']
    # Download all tiles for project area, process, and clip/merge
    for tile in tiles:
        file_name = tile + ".las"
        print("Processing LIDAR tile " + tile)
        tile_path = os.path.join(
            data_dir,
            'earthpy-downloads',
            file_name
        )
        download_url = las_root_url + tile + ".las"
        if not os.path.exists(tile_path):
            et.data.get_data(url=download_url)
        # Use LASTools from command line via subprocess
        # Install LASTools from here https://rapidlasso.de/downloads/

        # Output path for first returns DEM
        dem_fr_path = os.path.join(
            project_dir,
            tile +'_fr.tif'
        )
        print(dem_fr_path)
        # Output path for ground DEM
        dem_gr_path = os.path.join(
            project_dir,
            tile +'_gr.tif'
        )
        # Process first returns
        #if not os.path.exists(dem_fr_path):
        try:
            command = [lastools_path, "-i", tile_path, "-o", dem_fr_path, "-first_only"]
            completed_process = subprocess.run(command, capture_output=True, text=True)
            print("STDOUT:")
            print(completed_process.stdout)
            print("STDERR:")
            print(completed_process.stderr)
        except subprocess.CalledProcessError as e:
            print("Error:", e)
        # Process ground returns
        #if not os.path.exists(dem_gr_path):
        #subprocess.run([lastools_path, "-i", tile_path, "-o", dem_gr_path, "-keep_class", "2"])
        #first_return_dem = rxr.open_rasterio(dem_fr_path)
        #ground_return_dem = rxr.open_rasterio(dem_gr_path)
        # Calculate canopy DEM
        #canopy_dem = first_return_dem - ground_return_dem
        
    
    

In [165]:
lastools_path

'C:\\Users\\Pete\\Desktop\\GIS\\LAStools\\bin\\las2dem.exe'

In [168]:
import subprocess

# Define the command to run
command = ["C:\\Users\\Pete\\Desktop\\GIS\\LAStools\\bin\\las2dem.exe", "-i", "C:\\Users\\Pete\\earth-analytics\\data\\earthpy-downloads\\N4W298.las", "-first_only", "-o", dem_fr_path]

# Run the command and capture the output
completed_process = subprocess.run(command, capture_output=True, text=True)

# Print the output
print("STDOUT:")
print(completed_process.stdout)
print("STDERR:")
print(completed_process.stderr)


STDOUT:

STDERR:
Please note that LAStools is not "free" (see http://lastools.org/LICENSE.txt)
contact 'info@rapidlasso.de' to clarify licensing terms if needed.
ERROR: no input specified
usage:
las2dem -i lidar.las -o lidar.asc
las2dem -i lidar.las -o lidar.bil -intensity -kill 50
las2dem -i *.las -step 2.0 -opng -hillshade
las2dem -i lidar.las -o lidar.png -utm 11S -false
las2dem -i lidar.las -o lidar.png -sp83 TX_N -intensity -false -set_min_max 10 50
las2dem -i lidar.las -ll 640000 4320000 -ncols 400 -nrows 400 -o lidar.jpg -gray
las2dem -i lidar.las -keep_class 2 -o dem.png -sp27 PA_N -last_only -gray
las2dem -i lidar.las -keep_class 8 3 -o dem.tif -step 2.0 -intensity
las2dem -h



In [4]:
# Download test DRAPP tile and use it to find overlapping LAS files

test_tile_path = os.path.join(
    data_dir,
    'earthpy-downloads',
    'N3W345.tif'
)

if not os.path.exists(test_tile_path):
    test_tile_url = 'https://drapparchive.s3.amazonaws.com/2020/N3W345.tif'
    et.data.get_data(url=test_tile_url)
    
test_aerial = rxr.open_rasterio(test_tile_path)

test_aerial = test_aerial.rio.set_crs("EPSG:4326")

#aerial_bounds = test_aerial.rio.bounds()

test_aerial_plot = test_aerial.hvplot.rgb(rasterize=True, aspect='equal', x = 'x', y = 'y',tiles = 'OSM')
test_aerial_plot

:DynamicMap   []
   :Overlay
      .Tiles.I :Tiles   [x,y]
      .RGB.I   :RGB   [x,y]   (R,G,B)

In [160]:
# Use LASTools from command line via python
# Install LASTools from here https://rapidlasso.de/downloads/
import subprocess
# Example: Run las2dem command with first returns
las_path = os.path.join(
    data_dir,
    'earthpy-downloads',
    'N3W345.las'
)
# Output path for first returns DEM
dem_fr_path = os.path.join(
    data_dir,
    'N3W345_fr.tif'
)
# Output path for ground DEM
dem_gr_path = os.path.join(
    data_dir,
    'N3W345_gr.tif'
)
# Set this to your LASTools directory after installing
lastools_path = "C:\\Users\\Pete\\Desktop\\GIS\\LAStools\\bin\\las2dem.exe"
# Process first returns
subprocess.run([lastools_path, "-i", las_path, "-o", dem_fr_path, "-first_only"])
# Process ground returns
subprocess.run([lastools_path, "-i", las_path, "-o", dem_gr_path, "-keep_class", "2"])
first_return_dem = rxr.open_rasterio(dem_fr_path)
ground_return_dem = rxr.open_rasterio(dem_gr_path)
# Calculate canopy DEM
canopy_dem = first_return_dem - ground_return_dem

In [6]:
first_return_dem.name = 'first_return_dem'
canopy_dem.name = 'canopy_dem_name'

In [14]:
canopy_dem = canopy_dem.where(canopy_dem >= 1, np.nan)
canopy_dem = canopy_dem.where(canopy_dem <= 500, np.nan)

In [17]:
canopy_dem = canopy_dem.rio.reproject("EPSG:4326")
canopy_dem.values.max()

nan

In [21]:
canopy_dem_plot = canopy_dem.hvplot(
    geo=True,
    rasterize=True,
    aspect='equal',
    kind='image',
    tiles = 'EsriImagery',
    alpha=0.5,
    title = "LIDAR Canopy Example",
    clabel= 'Height in feet')
canopy_dem_plot

:DynamicMap   [band]
   :Overlay
      .WMTS.I  :WMTS   [Longitude,Latitude]
      .Image.I :Image   [x,y]   (canopy_dem_name)

In [18]:
# Define input file path
input_las_file = las_path

# Open the LAS file
las = pylas.read(input_las_file)

# Get the first point data
first_point_data = next(iter(las.points_data), None)

# Print keys of the first point data
if first_point_data:
    print(dir(first_point_data))
else:
    print("No points data available.")

['T', '__abs__', '__add__', '__and__', '__array__', '__array_interface__', '__array_priority__', '__array_struct__', '__array_wrap__', '__bool__', '__class__', '__copy__', '__deepcopy__', '__delattr__', '__delitem__', '__dir__', '__divmod__', '__doc__', '__eq__', '__float__', '__floordiv__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__int__', '__invert__', '__le__', '__len__', '__lshift__', '__lt__', '__mod__', '__mul__', '__ne__', '__neg__', '__new__', '__or__', '__pos__', '__pow__', '__radd__', '__rand__', '__rdivmod__', '__reduce__', '__reduce_ex__', '__repr__', '__rfloordiv__', '__rlshift__', '__rmod__', '__rmul__', '__ror__', '__rpow__', '__rrshift__', '__rshift__', '__rsub__', '__rtruediv__', '__rxor__', '__setattr__', '__setitem__', '__setstate__', '__sizeof__', '__str__', '__sub__', '__subclasshook__', '__truediv__', '__xor__', 'all', 'any', 'argmax', 'argmin', 'argsort', 'astype', 'base', 'byteswap', 'cho

In [ ]:
first_return_points

In [ ]:
# Define input and output file paths
las_path = os.path.join(
    data_dir,
    'earthpy-downloads',
    'N3W345.las'
)
output_dem_file = os.path.join(
    data_dir,
    'test_fr.tif'
)

# Open the input .las file
in_las = laspy.read(las_path)

# Extract first return points
first_return_indices = np.where(in_las.return_num == 1)[0]
first_return_points = in_las.points[first_return_indices]


# Define the fraction of points to retain (e.g., 0.5 for 50%)
fraction_to_retain = 0.25

# Calculate the number of points to retain
num_points_to_retain = int(len(first_return_points) * fraction_to_retain)

# Randomly select indices of points to retain
indices_to_keep = np.random.choice(len(first_return_points), size=num_points_to_retain, replace=False)

# Select the retained points
retained_points = first_return_points[indices_to_keep]

# Extract x, y, z coordinates
x = retained_points['X']
y = retained_points['Y']
z = retained_points['Z']

# Define grid parameters
xmin, xmax = np.min(x), np.max(x)
ymin, ymax = np.min(y), np.max(y)
resolution = 1000  # Adjust resolution as needed

# Create grid
grid_x, grid_y = np.mgrid[xmin:xmax:resolution, ymin:ymax:resolution]
grid_z = griddata((x, y), z, (grid_x, grid_y), method='linear')

# Write grid to GeoTIFF
transform = from_origin(xmin, ymax, resolution, resolution)
crs = vlrs[0].parse_crs()
with rasterio.open(output_dem_file, 'w', driver='GTiff', height=grid_z.shape[0], width=grid_z.shape[1], count=1, dtype=grid_z.dtype, crs=crs, transform=transform) as dst:
    dst.write(grid_z, 1)

In [34]:
crs = vlrs[0].parse_crs()
with rasterio.open(output_dem_file, 'w', driver='GTiff', height=grid_z.shape[0], width=grid_z.shape[1], count=1, dtype=grid_z.dtype, crs=crs, transform=transform) as dst:
    dst.write(grid_z, 1)

In [33]:
crs = vlrs[0].parse_crs()
crs

<Compound CRS: COMPD_CS["NAD83(2011) / Colorado North (ftUS) + NA ...>
Name: NAD83(2011) / Colorado North (ftUS) + NAVD88 height - Geoid18 (ftUS)
Axis Info [cartesian|vertical]:
- [east]: X (US survey foot)
- [north]: Y (US survey foot)
- [up]: Up (US survey foot)
Area of Use:
- undefined
Datum: NAD83 (National Spatial Reference System 2011)
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich
Sub CRS:
- NAD83(2011) / Colorado North (ftUS)
- NAVD88 height (ftUS)

In [ ]:
plot = dem.hvplot(
    x='x',
    y='y',
    rasterize=True,
    #cmap='viridis',
    aspect='equal',
    clim=(dem.min(), dem.max()),
    title="DEM of First Returns",
    xlabel="Longitude",
    ylabel="Latitude",
    crs=4326
)
plot

In [51]:
# Download and process LAS file
las_path = os.path.join(
    data_dir,
    'earthpy-downloads',
    'N3W345.las'
)

if not os.path.exists(las_path):
    las_url = 'https://lidararchive.s3.amazonaws.com/2020_CSPN_Q2/N3W345.las'
    et.data.get_data(url=las_url)

in_las = laspy.read(las_path)

first_returns = in_las.points[in_las.return_num == 1]

# Randomly sample a fraction of the points (adjust sample_fraction as needed)
sample_fraction = 0.1  # 10% of the points
sample_indices = np.random.choice(range(len(first_returns)), size=int(len(first_returns) * sample_fraction), replace=False)
x = first_returns['X'][sample_indices]
y = first_returns['Y'][sample_indices]
z = first_returns['Z'][sample_indices]/1000

# Define grid resolution
grid_resolution = 1000  # Adjust as needed
#grid_downsample_factor = 100  # Adjust as needed

# Downsample the grid
x_grid = np.arange(min(x), max(x), grid_resolution)
y_grid = np.arange(min(y), max(y), grid_resolution)
#x_grid_downsampled = x_grid[::grid_downsample_factor]
#y_grid_downsampled = y_grid[::grid_downsample_factor]
xx, yy = np.meshgrid(x_grid, y_grid)

# Interpolate elevation values onto the downsampled grid
zi = griddata((x, y), z, (xx, yy), method='linear')

# Convert zi to an xarray.DataArray
crs = in_las.vlrs[0].parse_crs
coords = {'x': x_grid, 'y': y_grid}
zi_xarray = xr.DataArray(zi, coords=coords, dims=('y', 'x'))

# Convert zi_xarray to a rioxarray.DataArray
#zi_rioxarray = rxr.open_rasterio(zi_xarray, crs=crs)

# # Visualize the DEM
# plt.imshow(zi, extent=(min(x), max(x), min(y), max(y)))
# plt.colorbar(label='Elevation (m)')
# plt.xlabel('X')
# plt.ylabel('Y')
# plt.title('Digital Elevation Model')
# plt.show()
# dem_array = rxr.open_rasterio(zi)

# dem_array.hvplot(rasterize=True,
#         aspect='equal')

In [63]:
zi_xarray = xr.DataArray(zi, coords=coords, dims=('y', 'x'))
crs = in_las.vlrs[0].parse_crs()
zi_xarray.rio.set_crs(crs)
zi_xarray.rio.reproject("EPSG:4326")
# crs = in_las.vlrs[0].parse_crs()
# zi_xarray = zi_xarray.rio.reproject("EPSG:4326")

# #zi_xarray = zi_xarray.rio.reproject("EPSG:4326")

zi_xarray.hv.QuadMesh(
    rasterize=True,
    x='x',
    y='y',
    crs=4326
)
    

AttributeError: 'DataArray' object has no attribute 'hv'

In [3]:
las_path = os.path.join(
    data_dir,
    'earthpy-downloads',
    'N3W345.las'
)
output_dem_file = os.path.join(
    data_dir,
    'test_fr.tif'
)


# Open the LAS file
infile = liblas.file.File(las_path, mode='r')

# Get header information
header = infile.header

# Get spatial reference
srs = osr.SpatialReference()
srs.ImportFromEPSG(header.srs.get_epsg())

# Filter for first returns
first_returns = []
for p in infile:
    if p.return_number == 1:
        first_returns.append((p.x, p.y, p.z))

# Convert to numpy array
first_returns = np.array(first_returns)

# Determine the grid dimensions and resolution
min_x = np.min(first_returns[:, 0])
max_x = np.max(first_returns[:, 0])
min_y = np.min(first_returns[:, 1])
max_y = np.max(first_returns[:, 1])
x_resolution = 1  # Adjust as needed
y_resolution = 1  # Adjust as needed
cols = int((max_x - min_x) / x_resolution) + 1
rows = int((max_y - min_y) / y_resolution) + 1

# Convert numpy array to xarray DataArray
coords = {'x': np.linspace(min_x, max_x, cols), 'y': np.linspace(min_y, max_y, rows)}
data_array = xr.DataArray(first_returns[:, 2].reshape(rows, cols), coords=coords, dims=('y', 'x'))

# Write the xarray DataArray to a GeoTIFF
data_array.rio.set_spatial_dims('x', 'y', inplace=True)
data_array.rio.to_raster(output_tif, crs=srs.ExportToWkt())

# Close the LAS file
infile.close()

NameError: name 'liblas' is not defined